In [9]:
#XINGYU CHEN XC374,  BYRAN MIN km567
#preprocessing 
import sys
import os
import numpy as np
import re
from num2words import num2words
from nltk import word_tokenize
import re
import pandas as pd
import math
from nltk import word_tokenize


def preprocess(filename):
    f=open(filename,'r',encoding="utf8")
    content=f.readlines()
    #strip() to delete extra space and  new line symbols
    content = " ".join(x.strip().strip('\n').strip('\t') for x in content)
    #re-assemble words like there ' s , they 've, don't,can't into a consitent form 
    
    

    content= re.sub(r" \’ ",r"'",content)
    content= re.sub(r" \'",r"'",content)
    content= re.sub(r"\' ",r"'",content)
    content=re.sub(r" n\’t",r"n't",content)
    

    #replace puntuation into period
    #content=re.sub(r"[/!/?]",r' . ',content)
    content=re.sub(r"''",r" ",content)
    content=re.sub(r"“",r" ",content)
    content=re.sub(r"`","",content)
    content=re.sub(r"\.\.\.",r"",content)
    content=re.sub(r"[\-\—\…]",r" ",content)
    content=re.sub(r"\[",r"",content)
    content=re.sub(r"\]",r"",content)


    #replace puntuation into period
    #content=re.sub(r"[/!/?]",r' . ',content)


    def is_number(text):
        try:
            float(text)
            return True
        except ValueError:
            return False
    def conert_number2string(text):

        t=text.split(" ")
        c=[ num2words((float)(a)) if is_number(a) else a for a in t ]
        return " ".join(c)
    #replace number into word characters
    content=conert_number2string(content)
    
    content=re.sub(r"\.",r" . ",content)
    #content=re.sub(r"\'",r" ' ",content)
    content=re.sub(r"\,",r" , ",content)
    #add start / end token
    #content=re.sub(r"\.",r"<s> . </s>",content)
        #add start / end token
    content=re.sub(r"\.",r". </s> <s> ",content)
    content=re.sub(r"\!",r"! </s> <s> ",content)
    content=re.sub(r"\?",r"? </s> <s> ",content)
    content='<s> '+ content
    content=content[:len(content)-4]
    content=re.sub(r"\.",r" ",content)
    #tokenization with re by space
    tokens = re.split(' ', content)
    tokens= list(filter(None, tokens)) # Remove empty strings
    return tokens

#this function generates N gram for any N, 98j kj
#returns total number of ngram
#returns vocab for the ngram models
#returns a word_dict for all the seen ngrams
# returns a sorted probality 
def generateNgram(tokens,n):
    ngram = [tuple(tokens[i:i+n]) for i in range(len(tokens)-(n-1))]
    ngram_count=len(ngram)
    word_dict = dict()
    for token in ngram:
        if token in word_dict.keys():
            word_dict[token] += 1
        else:
            word_dict[token] = 1 
    
    word_list_sorted=sorted(word_dict.items(),key=lambda x: (-x[1]))
    word_dict_sorted=dict(word_list_sorted)
    return ngram_count,word_dict_sorted,word_list_sorted


def loaddata(file):
    f=open(file,'r',encoding="utf8")
    content=f.readlines()

    content = " ".join(x.strip().strip('\n').strip('\t') for x in content)
    return content
def tokennize(content):
    
    content= re.sub(r" \’ ",r"'",content)
    content= re.sub(r" \'",r"'",content)
    content= re.sub(r"\' ",r"'",content)
    content=re.sub(r" n\’t",r"n't",content)
    

    #replace puntuation into period
    #content=re.sub(r"[/!/?]",r' . ',content)
    content=re.sub(r"''",r" ",content)
    content=re.sub(r"“",r" ",content)
    content=re.sub(r"`","",content)
    content=re.sub(r"\.\.\.",r"",content)
    content=re.sub(r"[\-\—\…]",r" ",content)
    content=re.sub(r"\[",r"",content)
    content=re.sub(r"\]",r"",content)


    def is_number(text):
        try:
            float(text)
            return True
        except ValueError:
            return False
    def conert_number2string(text):

        t=text.split(" ")
        c=[ num2words((float)(a)) if is_number(a) else a for a in t ]
        return " ".join(c)
    #replace number into word characters
    content=conert_number2string(content)
     
    content=re.sub(r"\.",r".",content)
    #content=re.sub(r"\'",r" ' ",content)
    content=re.sub(r"\,",r" , ",content)
    
     #add start / end token
    content=re.sub(r"\.",r". </s> <s> ",content)
    content=re.sub(r"\!",r"! </s> <s> ",content)
    content=re.sub(r"\?",r"? </s> <s> ",content)
    content='<s> '+ content
    content=content[:len(content)-4]
    #tokenization with re by space
    content=re.sub(r"\.",r" ",content)
    tokens = re.split(' ', content)
    tokens= list(filter(None, tokens)) # Remove empty strings
    return tokens



def wordcount(tokens):
    word_dict = dict()
    for token in tokens:
        if token in word_dict.keys():
            word_dict[token] += 1
        else:
            word_dict[token] = 1 
    word_list_sorted=dict(sorted(word_dict.items(),key=lambda x: (x[1])))

    return word_list_sorted

def merge_two_dicts(x, y):
    z = x.copy()   # start with x's keys and values
    z.update(y)    # modifies z with y's keys and values & returns None
    return z



#for obama corpus
#add unkown token :UNK" to the dictonary


def addunkown(dictionary,vocab):
    count=0
    keylist=list(dictionary.keys())
    for k in keylist:
        if k in vocab:
            continue
        else:
            count+=dictionary[k]
            del dictionary[k]

    for k in vocab:
        if k not in list(dictionary.keys()):
            dictionary[k]=0       
    dictionary['UNK']=count
    #start word probabilty is zero 
    dictionary['<s>']=1.0
    return dictionary


#amp the unknown tokens on corpus 
def maptoken(dictionary,tokens):
    for i in range(len(tokens)):
        try:
            dictionary[tokens[i]]
        except:
            tokens[i]="UNK"
    return tokens

                                                         


#apply smoothing
import math
def UnigramProbAdd1(n_vocab,vocab_dict,k):
    
    V = n_vocab
    N = sum(vocab_dict.values())

    prob_dict=dict()
    for key,value in vocab_dict.items():
        #unigram for key
   

        prob = ( value + k ) / ( N+k*V)
        
        prob_dict[key]=prob
    #start word probabilty is zero for unigram
    prob_dict['<s>']=1.0
    prob_dict=dict(sorted(prob_dict.items(),key=lambda x: (-x[1])))
    return prob_dict

def BigramProbAdd1(n_vocab, unigram_dict,bigram_dict,k):
    
    V = n_vocab
    prob_dict=dict()
    for key,value in bigram_dict.items():
        #unigram for key
        prob = ( bigram_dict[key] + k ) / (unigram_dict[key[0]] + k*V)
        
        prob_dict[key]=prob
    #corrent teh probabilty for </s> <s>
    prob_dict[('</s>', '<s>')]=1.0
    return prob_dict






def UnigramPerplexity(text,prob_dict,vocab_size):
    prob=0
    s=0
    V=vocab_size
    for token in text:
 
        if(token=='<s>'):
            s+=1
        log_prob=math.log(prob_dict[token])
       
        #print(prob_dict[token])
        prob+=(-log_prob)
    #delete the start token
    n=len(text)-s
    prob=prob/n
    
    return math.exp(prob)
#test the perplexity of devoplopment set
def BigramPerplexity(text,prob_dict,unigram_dict,vocab_size,k):
    prob=0
    bigram = [tuple(text[i:i+2]) for i in range(len(text)-(1))]
   # print(bigram)
   
    V=vocab_size
    s=0

    
    for token in bigram:
        try:
           if(token==('</s>', '<s>')):
                s+=1
     
           log_prob=math.log(prob_dict[token])
            #print(prob_dict[token])
        #if it's not in the dictionary, assign a probabily of 1/N+V   N is the count of uigram and V is to total vocalbuary
        except:
           
            log_prob=math.log(k/(unigram_dict[token[0]]+V))
            #print(1/(unigram_dict[token[0]]+V))
    
        prob+=-log_prob
    n=len(text)-s-1
    prob=prob/n
    #print(s)
    return math.exp(prob)
def findacc(truth,preds): 
     acc=np.sum(truth==preds)/len(truth)
     return acc
def processtest(file):
    f=open(file,'r',encoding="utf8")
    content=f.readlines()    
    #content=[x.strip().strip('\n').strip('\t') for x in content]
    content=[tokennize(x.strip()) for x in content]
    return content



#extract the key whose value is large than 1
def getvocab(dict1,dict2):
    vocab1=[i for i,x in dict1.items() if x>0]
    vocab2=[i for i,x in dict2.items() if x>0]

    vocab=list(set(vocab1)&set(vocab2))

    t=list(set(vocab1)-set(vocab))
    a=list(set(vocab2)-set(vocab))
    print(len(t))
   # print(t)
   # print(len(a))
    vocab1=[i for i in t if dict1[i]>=2]
    print(len(vocab1))
    vocab2=[i for i in a if dict2[i]>=2]
    print(len(vocab2))
    vocab=list(set(vocab)|set(vocab1)|set(vocab2))
    
    #print(len(vocab))
    #print(len(vocab1))
    #print(len(vocab2))
    #print(vocab_size)
    l1=len(vocab1)
    l2=len(vocab2)
    vocab1=[i for i in t if dict1[i]==1 ][:int(0.98*l1)]
    vocab2=[i for i in a if dict2[i]==1 ][:int(0.98*l2)]
    #print(len(vocab1))
    #print(len(vocab2))
    vocab=list(set(vocab)|set(vocab1)|set(vocab2))
   # vocab_size=len(vocab)
    print(len(vocab))
    return vocab
#merge two to be as our fixed vocab to estimate unkown words in the training corpus for two datasets

import random

def weighted_choice(choices):
    #choice is set of the all the possible words given by a preceding sequence of words 
    #for bigram that is all the possible words followed by a specfic word
   if choices==[]:
      return [None]
   #sum all the count in the choices set
   total = sum(w for c, w in choices)
   
   #gerate a random number in the range of (0, total_count)
   r = random.uniform(0, total)
   upto = 0
   # iterate over all the choices of  words
   for c, w in choices:
      if upto + w > r:
         #once the cumulative prob exceeds the generated count r, we return the word
         #print(c)
         return c
      upto += w

#method 'naive' means to always generate the highest probabilty word (the word has the highest count given its preceding word)
#method 'weighted' means to generate diversified sentences based on a weighted distribution of ngram counts 
#this function 
def generateNGramSentence( start,word_dict_sorted,n):
    sentence=list(start)
    words=start
    word=start[-1]
    # for unigram model 
    

    #for other ngram model (n>=2)

    while(1):
       # if (method=='naive'):
            #word=next((a[0][-1]  for a in word_dict_sorted if a[0][:n-1] ==words[-(n-1):]),'/s')     
         #   word=next((a[-1]for a in word_dict_sorted.keys()  if a[:n-1] ==words[-(n-1):]),None)

        if(n==1):
            word=weighted_choice(list(word_dict_sorted.items()))[-1]
            
            #r=random.randint(0,len(word_dict_sorted.keys())-1)
            #word=list(word_dict_sorted.keys())[r][0]
        else:
            choices=[(a,word_dict_sorted[a])   for a in word_dict_sorted if a[:n-1] ==words[-(n-1):]]
            word=weighted_choice(choices)[-1]
        if(n==1 and word=='<s>'):
            continue
        sentence.append(word)
        #for unknown word we will not handled it here but will deal with it with smoothing 
        if(word==None):
           print("ngram not found")
           return 
       
        if(word=='</s>'):
           break
        words=words+tuple([word])
 
    if(sentence[-1]!='</s>'):
        sentence.append("</s>")

    return (" ".join(sentence))
def generateword( start,word_dict_sorted,n):
    sentence=list(start)
    words=start
    word=start[-1]
    # for unigram model 
    

    #for other ngram model (n>=2)
    while(1):
        if(n==1):
            word=weighted_choice(list(word_dict_sorted.items()))[-1]

            #r=random.randint(0,len(word_dict_sorted.keys())-1)
            #word=list(word_dict_sorted.keys())[r][0]
        else:
            choices=[(a,word_dict_sorted[a])   for a in word_dict_sorted if a[:n-1] ==words[-(n-1):]]
            word=weighted_choice(choices)[-1]

            sentence.append(word)
            #for unknown word we will not handled it here but will deal with it with smoothing 


        if(word=='</s>'or word=='<s>'):
            continue
        else:
            break
    return word

def predict(a,b, t):
    pred=[]
    Id=[]
    result=[]
    for i,d in enumerate(test):
        tokens=maptoken(dict1,d)
        prediction=np.argmax([BigramPerplexity(tokens,bigram_trump,dict2,vocab_size,k),BigramPerplexity(tokens,bigram_obama,dict1,vocab_size,k)])
        result.append((i,prediction))
        pred.append(prediction)
    return pred,result
    

def run(text1,text2):
    tokens1=tokennize(text1)
    tokens2=tokennize(text2)
    #
    dict1=wordcount(tokens1)
    dict2=wordcount(tokens2)

    vocab=getvocab(dict1,dict2)

    dict1=addunkown(dict1,vocab)
    dict2=addunkown(dict2,vocab)
    tokens1=maptoken(dict1,tokens1)
    tokens2=maptoken(dict2,tokens2)

    #generate new bigram dicts
    bigram_count1,bi_dict1,bigram_sorted_list1=generateNgram(tokens1,2)
    bigram_count2,bi_dict2,bigram_sorted_list2=generateNgram(tokens2,2)
    n_vocab=len(vocab)     
    k=1
    unigram_obama=UnigramProbAdd1(n_vocab,dict1,k) #
    unigram_trump=UnigramProbAdd1(n_vocab,dict2,k)
    bigram_obama=BigramProbAdd1(n_vocab,dict1,bi_dict1,k)
    bigram_trump=BigramProbAdd1(n_vocab,dict2,bi_dict2,k)
    
    test_file="Assignment1_resources/test/test.txt"
    test=processtest(test_file)
    
    #unigram perplexity
    pred=[]
    Id=[]
    for i,d in enumerate(test):
        tokens=maptoken(dict1,d)
        prediction=np.argmax([UnigramPerplexity(tokens,unigram_trump,vocab_size),UnigramPerplexity(tokens,unigram_obama,vocab_size)])
        pred.append((i,prediction))
     
    labels = ['Id','Prediction']
    df = pd.DataFrame.from_records(pred, columns=labels)
    df.to_csv("unigram_1.csv",index=False,header=True,sep=",")
    print('done1')
    
    
    #bigram perplexity
    

    Id=[]
    pred=[]
    for i,d in enumerate(test):
        tokens=maptoken(dict1,d)
        prediction=np.argmax([BigramPerplexity(tokens,bigram_trump,dict2,vocab_size,k),BigramPerplexity(tokens,bigram_obama,dict1,vocab_size,k)])
        pred.append((i,prediction))
      
    labels = ['Id','Prediction']
    df = pd.DataFrame.from_records(pred, columns=labels)
    df.to_csv("bigram_1.csv",index=False,header=True,sep=",")
    print('done2')
    return 

In [22]:
#random sentence generation
filename1="Assignment1_resources/train/obama.txt"
filename2="Assignment1_resources/train/trump.txt"
obama_tokens=preprocess(filename1)
trump_tokens=preprocess(filename2)

obama_unigram_count,obama_unigram_dict,obama_unigram_sorted_list=generateNgram(obama_tokens,1)
trump_unigram_count,trump_unigram_dict,trump_unigram_sorted_list=generateNgram(trump_tokens,1)
obama_bigram_count,obama_bigram_dict,obama_bigram_sorted_list=generateNgram(obama_tokens,2)
trump_bigram_count,trump_bigram_dict,trump_bigram_sorted_list=generateNgram(trump_tokens,2)
#build probabilty dictionary for generating sentences
obama_unigram_prob_dict=dict([(i[0],i[1]/obama_unigram_count) for i in obama_unigram_sorted_list])
trump_unigram_prob_dict=dict([(i[0],i[1]/trump_unigram_count) for i in trump_unigram_sorted_list])
#build probabilty dictionary for generating sentences
obama_bigram_prob_dict=dict([(i[0],i[1]/obama_unigram_dict[tuple([i[0][0]])])  for i in obama_bigram_sorted_list])
trump_bigram_prob_dict=dict([(i[0],i[1]/trump_unigram_dict[tuple([i[0][0]])])  for i in trump_bigram_sorted_list])


In [43]:
start="<s> we're going to"
start=tuple(start.split(" "))
start2='<s>' 
start2=tuple(start2.split(" "))

In [44]:
print("Unigram")
print("seeded:")
print("obama:")
print(generateNGramSentence(start,obama_unigram_prob_dict,1))
print("trump:")
print(generateNGramSentence(start,trump_unigram_prob_dict,1))
#testing sentence generaton without applying smoothing 
#first is the unigram model
#unseeded sentence generation
print()
print()
print("unseeded:")
#method='weighted'
print("obama:")
print(generateNGramSentence(start2,obama_unigram_prob_dict,1))
print("trump:")
print(generateNGramSentence(start2,trump_unigram_prob_dict,1))

Unigram
seeded:
obama:
<s> we're going to who occasion , said of and serving demands billions and egypt and in days white rural we ; safety the effort consequences a went pacific and , absence about of acknowledge century out season an three first between control ” tears jobs even what after american the and men and pathway beliefs peace of hits can lines reason to to to still , because , they let arrayed the this but mr grounded in </s>
trump:
<s> we're going to and a i , out the i rancic talk think to get to i'm magnificent respect </s>


unseeded:
obama:
<s> safe differences one this diploma the oil up resoundingly want a very : </s>
trump:
<s> disgrace not one – an if right been from the it was save they we're a done predicting i he's </s>


In [45]:
print("seeded:")
print("obama:")
print(generateNGramSentence(start,obama_bigram_prob_dict,2))
print("trump:")
print(generateNGramSentence(start,trump_bigram_prob_dict,2))
print()
print()
print("unseeded:")
print("obama:")
print(generateNGramSentence(start2,obama_bigram_prob_dict,2))
print("trump:")
print(generateNGramSentence(start2,trump_bigram_prob_dict,2))


seeded:
obama:
<s> we're going to partner and , enforce rules , wild claims are gathered from the same sectarian tensions between , too </s>
trump:
<s> we're going to be one hundred billion dollars in real , and replaced with everybody wants to a truly signs of the people outside go around – and i want it or five people </s>


unseeded:
obama:
<s> and the challenge </s>
trump:
<s> and a clown </s>


In [46]:
#try the trigram model
obama_trigram_count,obama_trigram_dict,obama_trigram_sorted_list=generateNgram(obama_tokens,3)
trump_trigram_count,trump_trigram_dict,trump_trigram_sorted_list=generateNgram(trump_tokens,3)
obama_trigram_prob_dict=dict([(i[0],i[1]/obama_bigram_dict[i[0][0:2]])for i in obama_trigram_sorted_list])
trump_trigram_prob_dict=dict([(i[0],i[1]/trump_bigram_dict[i[0][0:2]])for i in trump_trigram_sorted_list])

print("seeded:")
print("obama:")
print(generateNGramSentence(start,obama_trigram_prob_dict,3))
print("trump:")
print(generateNGramSentence(start,trump_trigram_prob_dict,3))
print()
print()
print("unseeded:")
#genrate a word using unigram
word1_o=generateword(start2,obama_bigram_prob_dict,2)
word2_t=generateword(start2,trump_bigram_prob_dict,2)
start_o=start2+(word1_o,)
start_t=start2+(word2_t,)


print("obama:")
print(generateNGramSentence(start_o,obama_trigram_prob_dict,3))
print("trump:")
print(generateNGramSentence(start_t,trump_trigram_prob_dict,3))

seeded:
obama:
<s> we're going to be unleashed in the region that's not what i care about right now right now so that missions have the will to prevent terrorist attacks and saved according to one trillion over the last two days , i'll be blunt a lot slower than nascar </s>
trump:
<s> we're going to sign your life can be someplace else , chris , tonight we're closing up the arenas </s>


unseeded:
obama:
<s> and we're not going to be smart and efficient and more just , and to protect ourselves and for them and what will happen if we have always loved what they did when i first arrived on this goal </s>
trump:
<s> but i don't want them in jail for eight years and have to put in a rainy rough day </s>


In [50]:
#try the quadgram model
obama_quadgram_count,obama_quadgram_dict,obama_quadgram_sorted_list=generateNgram(obama_tokens,4)
trump_quadgram_count,trump_quadgram_dict,trump_quadgram_sorted_list=generateNgram(trump_tokens,4)
obama_quadgram_prob_dict=dict([(i[0],i[1]/obama_trigram_dict[i[0][0:3]]) for i in obama_quadgram_sorted_list])
trump_quadgram_prob_dict=dict([(i[0],i[1]/trump_trigram_dict[i[0][0:3]]) for i in trump_quadgram_sorted_list])

print("seeded:")
print("obama:")
print(generateNGramSentence(start,obama_quadgram_prob_dict,4))
print("trump:")
print(generateNGramSentence(start,trump_quadgram_prob_dict,4))
print()
print()

print("unseeded:")
word1_o=generateword(start2,obama_bigram_prob_dict,2)
word2_t=generateword(start2,trump_bigram_prob_dict,2)
start_o=start2+(word1_o,)
start_t=start2+(word2_t,)
word2_o=generateword(start_o,obama_trigram_prob_dict,3)
word2_t=generateword(start_t,obama_trigram_prob_dict,3)

start_o=start_o+(word2_o,)
start_t=start_t+(word2_t,)
print("obama:")
print(generateNGramSentence(start_o,obama_quadgram_prob_dict,4))
print("trump:")
print(generateNGramSentence(start_t,trump_quadgram_prob_dict,4))

seeded:
obama:
<s> we're going to move forward , and you don't necessarily see an improvement in standards of living for those asean countries </s>
trump:
<s> we're going to make our own products </s>


unseeded:
obama:
<s> ” bad things happen , we make them better , we keep going </s>
trump:
<s> not in terms of three hours because who wants three hours ? </s>


In [51]:
file1_trn="Assignment1_resources/train/obama.txt"
file2_trn="Assignment1_resources/train/trump.txt"
file1_dev="Assignment1_resources/development/obama.txt"
file2_dev="Assignment1_resources/development/trump.txt"

train1=loaddata(file1_trn)
train2=loaddata(file2_trn)

tokens1=tokennize(train1)
tokens2=tokennize(train2)
#
dict1=wordcount(tokens1)
dict2=wordcount(tokens2)

vocab=getvocab(dict1,dict2)

dict1=addunkown(dict1,vocab)
dict2=addunkown(dict2,vocab)
tokens1=maptoken(dict1,tokens1)
tokens2=maptoken(dict2,tokens2)

#generate new bigram dicts
bigram_count1,bi_dict1,bigram_sorted_list1=generateNgram(tokens1,2)
bigram_count2,bi_dict2,bigram_sorted_list2=generateNgram(tokens2,2)
n_vocab=vocab_size=len(vocab)   

7302
3265
713
12009


In [52]:

for k in [0.1,0.2,0.5,1,2,3,4,5,6]:
    unigram_obama=UnigramProbAdd1(n_vocab,dict1,k) #
    unigram_trump=UnigramProbAdd1(n_vocab,dict2,k)

    bigram_obama=BigramProbAdd1(n_vocab,dict1,bi_dict1,k)
    bigram_trump=BigramProbAdd1(n_vocab,dict2,bi_dict2,k)


    pred=[]
    result=[]
    Id=[]
    file1_dev="Assignment1_resources/development/obama.txt"
    file2_dev="Assignment1_resources/development/trump.txt"
    dev1=processtest(file1_dev)
    dev2=processtest(file2_dev)
    acc_set=[]
    x_dev=np.concatenate((dev1,dev2),axis=0)
    y_dev=np.concatenate((np.zeros(100),np.ones(100)),axis=0)
    y_dev=np.array(y_dev,dtype=int)
    
    for i,d in enumerate(x_dev):
        tokens=maptoken(dict1,d)
        prediction=np.argmax([BigramPerplexity(tokens,bigram_trump,dict2,vocab_size,k),BigramPerplexity(tokens,bigram_obama,dict1,vocab_size,k)])
        result.append((i,prediction))
        pred.append(prediction)
    acc=findacc(y_dev,pred)
    acc1=findacc(y_dev[:100],pred[:100])
    acc2=findacc(y_dev[100:200],pred[100:200])
    print("k=%f:"%(k)+str(acc)+" "+str(acc1)+ " "+str(acc2))
    acc_set.append(acc)
    
print("done")

k=0.100000:0.975 1.0 0.95
k=0.200000:0.975 1.0 0.95
k=0.500000:0.97 1.0 0.94
k=1.000000:0.965 0.99 0.94
k=2.000000:0.94 0.96 0.92
k=3.000000:0.9 0.89 0.91
k=4.000000:0.82 0.76 0.88
k=5.000000:0.71 0.61 0.81
k=6.000000:0.6 0.44 0.76
done


In [67]:
#evlutae correctness of perplexity on dev sets
k=1
unigram_obama=UnigramProbAdd1(n_vocab,dict1,k) #
unigram_trump=UnigramProbAdd1(n_vocab,dict2,k)

bigram_obama=BigramProbAdd1(n_vocab,dict1,bi_dict1,k)
bigram_trump=BigramProbAdd1(n_vocab,dict2,bi_dict2,k)

dev_tokens1=preprocess(file1_dev)
dev_tokens2=preprocess(file2_dev)
dev1=maptoken(dict1,dev_tokens1)
dev2=maptoken(dict2,dev_tokens2)
print("Unigram_obama")
print(UnigramPerplexity(dev1,unigram_obama,n_vocab))
print(UnigramPerplexity(dev2,unigram_obama,n_vocab))

print("Unigram_trump")
print(UnigramPerplexity(dev1,unigram_trump,n_vocab))
print(UnigramPerplexity(dev2,unigram_trump,n_vocab))


Unigram_obama
534.2267639491849
512.8591700210692
Unigram_trump
784.4244352872781
340.77709765247715


In [68]:
print("Bigram_obama")
print(BigramPerplexity(dev1,bigram_obama,dict1,n_vocab,1))
print(BigramPerplexity(dev2,bigram_obama,dict2,n_vocab,1))

print("Bigram_trump")
print(BigramPerplexity(dev1,bigram_trump,dict1,n_vocab,1))
print(BigramPerplexity(dev2,bigram_trump,dict2,n_vocab,1))

Bigram_obama
1395.8035033669266
1363.0926652025676
Bigram_trump
2607.196215006014
757.8420963590414


In [64]:
print("accuracy with k being 1")
for k in [1]:
    unigram_obama=UnigramProbAdd1(n_vocab,dict1,k) #
    unigram_trump=UnigramProbAdd1(n_vocab,dict2,k)

    bigram_obama=BigramProbAdd1(n_vocab,dict1,bi_dict1,k)
    bigram_trump=BigramProbAdd1(n_vocab,dict2,bi_dict2,k)


    pred=[]
    result=[]
    Id=[]
    file1_dev="Assignment1_resources/development/obama.txt"
    file2_dev="Assignment1_resources/development/trump.txt"
    dev1=processtest(file1_dev)
    dev2=processtest(file2_dev)
    x_dev=np.concatenate((dev1,dev2),axis=0)
    y_dev=np.concatenate((np.zeros(100),np.ones(100)),axis=0)
    y_dev=np.array(y_dev,dtype=int)
    
    for i,d in enumerate(x_dev):
        tokens=maptoken(dict1,d)
        t=BigramPerplexity(tokens,bigram_trump,dict2,vocab_size,k)
        o=BigramPerplexity(tokens,bigram_obama,dict1,vocab_size,k)
        prediction=np.argmax([t,o])
        result.append((i,prediction))
        pred.append(prediction)
 
    acc=findacc(y_dev,pred)
    acc1=findacc(y_dev[:100],pred[:100])
    acc2=findacc(y_dev[100:200],pred[100:200])
    print("bigtram k=%f:"%(k)+str(acc)+" "+str(acc1)+ " "+str(acc2))
    
    pred=[]

    for i,d in enumerate(x_dev):
        tokens=maptoken(dict1,d)
        t=UnigramPerplexity(tokens,unigram_trump,vocab_size)
        o=UnigramPerplexity(tokens,unigram_obama,vocab_size)
        prediction=np.argmax([t,o])
        result.append((i,prediction))
        pred.append(prediction)
 
    acc=findacc(y_dev,pred)
    acc1=findacc(y_dev[:100],pred[:100])
    acc2=findacc(y_dev[100:200],pred[100:200])
    
    print("unigram: k=%f:"%(k)+str(acc)+" "+str(acc1)+ " "+str(acc2))
    
    
    
print("done")

accuracy with k being 1
bigtram k=1.000000:0.965 0.99 0.94
unigram: k=1.000000:0.94 0.95 0.93
done


In [14]:
#now we know the best k=1
#combine the train and dev set
#run the prediction model

file1_trn="Assignment1_resources/train/obama.txt"
file2_trn="Assignment1_resources/train/trump.txt"
file1_dev="Assignment1_resources/development/obama.txt"
file2_dev="Assignment1_resources/development/trump.txt"

train1=loaddata(file1_trn)+" "+loaddata(file1_dev)
train2=loaddata(file2_trn)+" "+loaddata(file2_dev)

run(train1,train2)
print("predictions are recorded")

7207
3289
685
12003
done1
done2


In [101]:
#part 2： analogy
import numpy as np
import pandas as pd
import re
def read_embedding(filename):
    embed = {}
    for line in open(filename,encoding='utf8'):
        line = line.strip().split()
        if (len(line)!=301):
            continue
        try:
            embed[(line[0])] = np.array(list(map(float, line[1:])))
        except:
            pass
    print('[%s]\n\tEmbedding size: %d' % (filename, len(embed)), end='\n')
    return embed
def preprocessfile(filename):
    train_set=[]
    train_label=[]
    f=open(filename,'r',encoding="utf8")
    for line in f:
        line=line.strip()
        line=re.sub('[\t\n]', '',line)
        set=line.split(" ")

        train_set.append(set[0:3])
        train_label.append(set[-1])
    return np.array(train_set),np.array(train_label)


def mapembedding(train_file,embeded_dict):
    oov=[]
    result=[]
    a=1
    for i in range (len(train_file)):
          a=1
          c=[]
          for j in train_file[i]:
           
                try:
                    e=embed[j]

                except:
                    try:
                        e=embed[j.lower()]
                        
                    except:
                        oov.append(i)
                        a=0
                        continue
             
                c.append(e)
          if(a!=0):
                result.append(c)
        
    result =np.array(result)
        
    return result 
def processembedding(embed):
    result=[]
    for i in range(len(embed)):
        a=0
        a=-embed[i][0]+embed[i][1]+embed[i][2]
        result.append(a)
    return np.array(result)

def createvocab(embed_dict):
    vocab=np.array(list(embed_dict.keys()))
    vocab_dict={}
    for i,w in enumerate(vocab):
        vocab_dict[w]=i
    return vocab,vocab_dict

def createvocab_dict(vocab):
    vocab_dict={}
    for i,w in enumerate(vocab):
        vocab_dict[w]=i
    return vocab_dict

def createnumpyembedding(embed_dict):
    return np.array(list(embed_dict.values()))
#print(x_train.shape)

def findacc(truth,preds):
     
     acc=np.sum(truth==preds)/len(truth)
     return acc
def cosineimilarity(e,v):
  
    #Input:
    #e = nxd input matrix with n row-vectors of dimensionality d (n is number of dictionary_keys)
    #v = mxd input matrix with m row-vectors of dimensionality d (m is number of test samples)
    # Output:
    # Matrix D of size nxm
    # s(i,j) is the cosinesimiarlity of embed(i,:) and test(j,:)
    g=e.dot(v.T)
    b=np.expand_dims(np.linalg.norm(e,axis=1),1)
    a=np.expand_dims(np.linalg.norm(v,axis=1),1)
    s=np.divide(g,np.multiply(b,a.T))
    # ... until here
    return s


def findknn(D,k):
    """
   # D=cos_distance matrix
   # k = number of nearest neighbors to be found
    
   # Output:
   # indices = kxm matrix, where indices(i,j) is the i^th nearest neighbor of xTe(j,:)
   # dists = Euclidean distances to the respective nearest neighbors
    """

    m = D.shape[1]
    ind = np.argsort(D, axis=0)
    indices = ind[-k:]
    r = np.array([_ for _ in range(m)], dtype=np.int)
    r = np.array([r] * k)
    
    dists = D[indices, r]

    return indices, dists
def vocab_third_col(x):
    third_col=x[:,-1]
    third_col_y=[]
    for e in third_col:
        try:
            t=vocab_dict[e]
        except:
            try:
                t=vocab_dict[e.lower()]
            except:
                continue
        third_col_y.append(t)
    third_col_y=np.array(third_col_y)
    return third_col_y  

#a is the result of nn classifer, b is the third_colmn vocab_index
def modify(a,b):
    d=np.ones((1,a.shape[1]),dtype=int)
    for i in range (a.shape[1]):
        if b[i]!=a[-1][i]:
            d[0][i]=a[-1][i]
        else:
            d[0][i]=a[-2][i]

    return d

In [97]:

x,y=preprocessfile(filename)
vocab=set(x.flatten())|set(y)
print("vocab_size:"+str(len(vocab)))

vocab_size:659


In [56]:
#load this file to create vocab
#embed_file="D:/glove.6B.300d.txt"
#embed=read_embedding(embed_file)
#vocab,vocab_dict=createvocab(embed)
#vocab=set(vocab)|set(x.flatten())|set(y)
#print("vocab_size:"+str(len(vocab)))


[D:/glove.6B.300d.txt]
	Embedding size: 400000


vocab_size:400379


In [59]:
#the load the embedding file which is 840B the large dataset
embed_file="D:/word_embedding/glove.840B.300d.txt"
embed=read_embedding(embed_file)
print("finish loading embedding file")

[D:/word_embedding/glove.840B.300d.txt]
	Embedding size: 2195875
finish loading embedding file


In [110]:
#create the 840B vesion of embedding based on the customized vocab
customized_dict={}
for v in vocab:
    try:
        customized_dict[v]=embed[v]
    except:
        #aslo we should consider decapitalized vocab
        try: 
            customized_dict[v.title()]=embed[v.title()]
        except:     
            pass

vocab,vocab_dict=createvocab(customized_dict)

embeddings=createnumpyembedding(customized_dict)
print("done")

done


In [111]:

x_train=processembedding(mapembedding(x,customized_dict))

In [112]:
#this is for convinientence of testing we save the vocab and embedding file locally
#np.save("840B_glove_vocab.npy",vocab)
#np.save("840B_glove_embeddings.npy",embeddings)

#uncomment below to load them 
#vocab=np.load("840B_glove_vocab.npy")
#embeddings=np.load("840B_glove_embeddings.npy")
#vocab_dict=createvocab_dict(vocab)
#customized_dict={}
#for i,v in enumerate(vocab):
 #       customized_dict[v]=embeddings[i]
    


In [113]:
import timeit
start_time = timeit.default_timer()

#go over the entrie text file
#break file into submatrixes
#each chunk contains 100 samples of data
#evaluate accuracy
truth=[vocab_dict[w] for w in y]
number=200
n_chunk=(math.ceil(len(x_train)/number))
e=embeddings
preds1=np.array([[]])
preds2=np.array([[]])
third_col=vocab_third_col(x)
print("starting searching the most similar word for test file")
for i in range (n_chunk):
    print("running the chunk %d..."%(i))
    v=x_train[number*(i):number*(i+1)]
    b=third_col[number*(i):number*(i+1)]
    s=cosineimilarity(e,v)
    pred,dist= findknn(s,2)
    preds1=np.concatenate((preds1,np.expand_dims(pred[-1],0)),axis=1)
    preds2=np.concatenate((preds2,modify(pred,b)),axis=1)
truth=np.array(truth).astype(int)
preds1=np.array(preds1).astype(int).flatten()
preds2=np.array(preds2).astype(int).flatten()
print("complete")
elapsed = timeit.default_timer() - start_time
print("running time:"+str(elapsed))

starting searching the most similar word for test file
running the chunk 0...
running the chunk 1...
running the chunk 2...
running the chunk 3...
running the chunk 4...
running the chunk 5...
running the chunk 6...
running the chunk 7...
running the chunk 8...
running the chunk 9...
running the chunk 10...
running the chunk 11...
running the chunk 12...
running the chunk 13...
running the chunk 14...
running the chunk 15...
running the chunk 16...
running the chunk 17...
running the chunk 18...
running the chunk 19...
running the chunk 20...
running the chunk 21...
running the chunk 22...
running the chunk 23...
running the chunk 24...
running the chunk 25...
running the chunk 26...
running the chunk 27...
running the chunk 28...
running the chunk 29...
running the chunk 30...
running the chunk 31...
running the chunk 32...
running the chunk 33...
running the chunk 34...
running the chunk 35...
running the chunk 36...
running the chunk 37...
running the chunk 38...
running the chunk 3

In [114]:
print("running time:"+str(elapsed)+" s")

running time:0.7977027039814857 s


In [121]:
correct=truth[truth==preds1]
correct_i=np.argwhere(truth==preds1).flatten()
false=preds1[truth!=preds1]
false_i=np.where([truth!=preds1])[1]
def randomdisplay(x,y,vocab,vocab_index,data_index,n):
    n_samples_for_display=n
    a=np.random.choice(len(correct),n_samples_for_display,replace=False)
    if(y[data_index[a[0]]]==vocab[vocab_index[a[0]]]):
        status="correct"
    else:
        status="incorrect"
    for i in range (n_samples_for_display):
        print("data:{:40s}".format((str(x[data_index[a[i]]]))) +"label:{:20s}".format((y[data_index[a[i]]]))
              +"pred:{:20s}".format(vocab[vocab_index[a[i]]])+"result: %s"%(status))
        print()
        
print("10 correct examples:")
randomdisplay(x,y,vocab,correct,correct_i,10)


10 correct examples:
data:['Harare' 'Zimbabwe' 'Niamey']          label:Niger               pred:Niger               result: correct

data:['Belgrade' 'Serbia' 'Kathmandu']       label:Nepal               pred:Nepal               result: correct

data:['USA' 'dollar' 'India']                label:rupee               pred:rupee               result: correct

data:['son' 'daughter' 'he']                 label:she                 pred:she                 result: correct

data:['Jakarta' 'Indonesia' 'Kathmandu']     label:Nepal               pred:Nepal               result: correct

data:['Colombia' 'Colombian' 'Denmark']      label:Danish              pred:Danish              result: correct

data:['simple' 'simpler' 'hard']             label:harder              pred:harder              result: correct

data:['Sofia' 'Bulgaria' 'Zagreb']           label:Croatia             pred:Croatia             result: correct

data:['Glendale' 'California' 'Louisville']  label:Kentucky            pred

In [123]:
print("10 false examples:")
randomdisplay(x,y,vocab,false,false_i,10)

10 false examples:
data:['mouse' 'mice' 'onion']                label:onions              pred:onion               result: incorrect

data:['Houston' 'Texas' 'Tulsa']             label:Oklahoma            pred:Texas               result: incorrect

data:['boy' 'girl' 'groom']                  label:bride               pred:groom               result: incorrect

data:['he' 'she' 'groom']                    label:bride               pred:groom               result: incorrect

data:['Dallas' 'Texas' 'Lexington']          label:Kentucky            pred:Lexington           result: incorrect

data:['Bakersfield' 'California' 'Irving']   label:Texas               pred:Irving              result: incorrect

data:['groom' 'bride' 'grandpa']             label:grandma             pred:grandpa             result: incorrect

data:['eagle' 'eagles' 'building']           label:buildings           pred:building            result: incorrect

data:['computer' 'computers' 'elephant']     label:elephants 

In [71]:
print("original model acc:")
print(findacc(truth,preds1))
print("modified model acc:")
print(findacc(truth,preds2))

0.375458951387869


In [124]:
word1='small'
word2='big'
word3='short'
print("customized analogy pair:"+str(word1)+" "+ str(word2)+" "+str(word3))
xx=-embed[word1]+embed[word2]+embed[word3]
xx=np.expand_dims(xx,0)
s=cosineimilarity(embeddings,xx)
pred,dist= findknn(s,1)
print("prediction:")
print(vocab[pred][0][0])

print()
word1='like'
word2='fancy'
word3='dislike'
print("customized analogy pair:"+str(word1)+" "+ str(word2)+" "+str(word3))
xx=-embed[word1]+embed[word2]+embed[word3]
xx=np.expand_dims(xx,0)
s=cosineimilarity(embeddings,xx)
pred,dist= findknn(s,1)
print("prediction:")
print(vocab[pred][0][0])

word='enter'
print("find 10 nearest neigbour of a word"+str(word))
xx=embed[word]
xx=np.expand_dims(xx,0)
s=cosineimilarity(embeddings,xx)
pred,dist= findknn(s,11)
print("prediction:(left to right from most similar to less similar)")
print(vocab[pred][:-1,:][::-1].flatten())

customized analogy pair:small big short
prediction:
short

customized analogy pair:like fancy dislike
prediction:
usual
find 10 nearest neigbour of a wordenter
prediction:(left to right from most similar to less similar)
['possible' 'quickly' 'safely' 'immediate' 'possibly' 'complete' 'longer'
 'usually' 'free' 'her']


In [126]:

word='enter'
print("find 10 nearest neigbour of a word"+str(word))
xx=embed[word]
xx=np.expand_dims(xx,0)

start_time = timeit.default_timer()
s=cosineimilarity(embeddings,xx)
pred,dist= findknn(s,2)
elapsed = timeit.default_timer() - start_time
print("running time:"+str(elapsed))

print("prediction:(left to right from most similar to less similar)")
print(vocab[pred][:-1,:][::-1].flatten())




find 10 nearest neigbour of a wordenter
running time:0.002250558798550628
prediction:(left to right from most similar to less similar)
['possible']


In [ ]:
#elmo vector/allennlp
import allennlp
from allennlp.modules.elmo import Elmo, batch_to_ids
import torch
options_file = "elmo_2x4096_512_2048cnn_2xhighway_5.5B_options.json"
weight_file = "elmo_2x4096_512_2048cnn_2xhighway_5.5B_weights.hdf5"
file1_trn="train/obama.txt"
file2_trn="train/trump.txt"
file1_dev="development/obama.txt"
file2_dev="development/trump.txt"
test_file="test/test.txt"

#precocess and load file
f=open(file1_trn,'r',encoding="utf8")
content1=f.readlines()
f=open(file1_dev,'r',encoding="utf8")
content2=f.readlines()
a=[preprocess(i) for i in content1]
b=[preprocess(i) for i in content2]
a=np.expand_dims(a,0)
b=np.expand_dims(b,0)
t=np.concatenate((a, b), axis=1)
trn1=list(t[0])
#
f=open(file2_trn,'r',encoding="utf8")
content1=f.readlines()
f=open(file2_dev,'r',encoding="utf8")
content2=f.readlines()
a=[preprocess(i) for i in content1]
b=[preprocess(i) for i in content2]
a=np.expand_dims(a,0)
b=np.expand_dims(b,0)
t=np.concatenate((a, b), axis=1)
trn2=list(t[0])

#tokenization
obama_tokens=[word_tokenize(i) for i in trn1]
trump_tokens=[word_tokenize(i) for i in trn2]
f=open(test_file,'r',encoding="utf8")
content1=f.readlines()
a=[preprocess(i) for i in content1]
a=np.expand_dims(a,0)
test=list(a[0])
test_tokens=[word_tokenize(i) for i in test]

def run_elmo(tokens,flag):
    n_chunk=math.ceil((len(tokens))/100)

    #size=0
    size=0 (#total number of words in the document)

    for i in range (n_chunk):
        result=torch.zeros(1024,dtype=torch.double)
        result1=torch.zeros(1024,dtype=torch.double) #/total number
        chunk=tokens[i*100:(i+1)*100]
        for j,seq_tokens in enumerate(chunk):
              print(i,j)
              character_ids=batch_to_ids([seq_tokens])
              t=elmo(character_ids)
              a=torch.sum(t['mask'],dim=1,dtype=torch.double).unsqueeze(1)
              size+=t['elmo_representations'][1].size()[1]
              b=torch.sum(t['elmo_representations'][1],dim=1,dtype=torch.double)
              result1=torch.add(result1,b.squeeze(0))
              c=torch.div(b,a).squeeze(0)
              result=torch.add(result,c)
        result=result.detach().numpy()
        result1=result1.detach().numpy()
        if(flag==1):
            np.save('elmo/o1_%d.npy'%(i),result)
            np.save('elmo/o2_%d.npy'%(i),result1)
        if(flag==1):
            np.save('elmo/t1_%d.npy'%(i),result)
            np.save('elmo/t2_%d.npy'%(i),result1)
        #print(size)
    print("final:")
    print(size) #total number of words in the document
    print('done')
        
run_elmo(trump_tokens)
run_elmo(obama_tokens)

#obain elmo representation of each speech vector by taking average
a=np.zeros(1024)
for i in range (0,31):
    result=np.load("elmo/t1_"+str(n)+'.npy')
    a+=result
print(a)
        
b=np.zeros(1024)
for i in range (0,31):
    result=np.load("elmo/o1_"+str(n)+'.npy')
    b+=result
print(a)
        
total=0
results=torch.zeros(1024,dtype=torch.double)

#generate embedding file for text data
print(n_chunk)
for i in range (len(test_tokens)):
        character_ids=batch_to_ids([test_tokens[i]])
        t=elmo(character_ids)
        a=torch.sum(t['mask'],dim=1,dtype=torch.double).unsqueeze(1)
        b=torch.sum(t['elmo_representations'][1],dim=1,dtype=torch.double)
        c=torch.div(b,a)
        if(i!=0):
            c=torch.cat((prev,c),dim=0)     
        prev=c
np.save("elmo3/test.npy",c.detach().numpy())

#generate embedding file for development data
results=torch.zeros(1024,dtype=torch.double)
x_dev=np.concatenate((obama_tokens[3000:3100],trump_tokens[3000:3100]),axis=0)

for i in range (len(x_dev)):
        print(i)
        character_ids=batch_to_ids([x_dev[i]])
        t=elmo(character_ids)
        a=torch.sum(t['mask'],dim=1,dtype=torch.double).unsqueeze(1)
        b=torch.sum(t['elmo_representations'][1],dim=1,dtype=torch.double)
        c=torch.div(b,a)
        if(i!=0):
            c=torch.cat((prev,c),dim=0)     
        prev=c
        print(c.size())

results
np.save("elmo3/dev.npy",c.detach().numpy())        
        
        
trump_embedding=a/3100
obama_embedding=b/3100
test_embedding=np.load("elmo3/test.npy")
        
obama_embedding=np.expand_dims(obama_embedding,axis=0)
trump_embedding=np.expand_dims(trump_embedding,axis=0)


pred=[]
Id=[]
result=[]
for i in range(200):
    test=np.expand_dims(test_embedding[i],axis=0)

    #print(line_embedding.shape)
    prediction=np.argmax([cosineimilarity(obama_embedding,test),cosineimilarity(trump_embedding,test)])
    result.append((i,prediction))
    pred.append(prediction)
        
labels = ['Id','Prediction']
df = pd.DataFrame.from_records(result, columns=labels)
df.to_csv("elmo.csv",index=False,header=True,sep=",")
print('done')
        
        


In [21]:
#obain elmo representation of each speech vector by taking average
a=np.zeros(1024)
for i in range (0,30):
    result=np.load("elmo3/t1_"+str(i)+'.npy')
    a+=result

b=np.zeros(1024)
for i in range (0,30):
    result=np.load("elmo3/o1_"+str(i)+'.npy')
    b+=result

trump_embedding=a/3000
obama_embedding=b/3000

obama_embedding=np.expand_dims(obama_embedding,axis=0)
trump_embedding=np.expand_dims(trump_embedding,axis=0)
x_dev=np.load("elmo3/dev.npy")

pred=[]
Id=[]
result=[]

for i in range(200):
    dev=np.expand_dims(x_dev[i],axis=0)

    #print(line_embedding.shape)
    prediction=np.argmax([cosineimilarity(obama_embedding,dev),cosineimilarity(trump_embedding,dev)])
    result.append((i,prediction))
    pred.append(prediction)
print("elmo vector classficaiton acc:")
print(findacc(y_dev,pred))

elmo vector classficaiton acc:
0.865


In [ ]:
test_embedding.shape

In [ ]:
#calculate the perplexity of trumps and obama speech in the dev sets
loaddata(file1_dev)

In [167]:
import numpy as np
pred=[]
Id=[]
f=open("Assignment1_resources/test/test.txt",'r',encoding="utf8")
content=f.readlines()
len(content)
for i,d in enumerate(content):
    tokens=tokennize(d.strip())
    tokens=maptoken(dict1,tokens)
    prediction=np.argmax([BigramPerplexity(tokens,bigram_trump,dict2,vocab_size),BigramPerplexity(tokens,bigram_obama,dict1,vocab_size)])
    pred.append((i,prediction))
pred



TypeError: BigramPerplexity() missing 1 required positional argument: 'k'